In [1]:
from django import setup
setup()

In [2]:
from django.utils import timezone
from entries.models import Entry, Nutrient

In [3]:
e1 = Entry(when=timezone.now(), what="something 4", quantity=2, category="c", measure="g")

In [7]:
e1.save()
e1.id

1

In [8]:
i1 = Nutrient()

In [9]:
i1.label = "sugar"
i1.unit = "g"
i1.category = "i"
i1.quantity = 4

In [10]:
i1.entry = e1

In [11]:
i1.save()

In [12]:
e = Entry.objects.all()[0]

In [14]:
e.what

'something 4'

In [26]:
Nutrient.objects.filter(entry=6).count()

0

In [12]:
n = Nutrient()
e = Entry.objects.all()[2]

In [13]:
n.quantity = 3.2
n.unit = "kcal"
n.category = "o"
n.label = "running"
n.entry = e

In [15]:
n.save()

In [15]:
from django.db.models import Sum, Count
Nutrient.objects.filter(category="o").aggregate(Sum("quantity"))

{'quantity__sum': 3849.6800000000003}

In [9]:
Nutrient.objects.filter(category="i", label="Energy", unit="kcal").aggregate(Sum("quantity"))

{'quantity__sum': 8956.0}

In [19]:
Nutrient.objects.values("category").annotate(Count("category"))

<QuerySet [{'category': 'i', 'category__count': 692}, {'category': 'o', 'category__count': 9}]>

In [48]:
print(Nutrient.objects.filter(
        entry__category="c").extra({"hour":"a"}).values(
        "hour").order_by().annotate(Count("hour")).query)

FieldError: Cannot resolve keyword 'hour' into field. Choices are: category, entry, entry_id, id, label, quantity, unit

In [4]:
from django.db import models

In [31]:
class JSONField(models.TextField):
    """
    JSONField is a generic textfield that neatly serializes/un-serializes
    JSON objects seamlessly.
    Django snippet #1478

    example:
        class Page(models.Model):
            data = JSONField(blank=True, null=True)


        page = Page.objects.get(pk=5)
        page.data = {'title': 'test', 'type': 3}
        page.save()
    """

    # def from_db_value(self, value, expression, connection, context):
    #     return self.to_python(value)

    def to_python(self, value):
        print("to_python")
        if value == "":
            return None

        try:
            from django.utils import six
            if isinstance(value, six.string_types):
                return json.loads(value)
        except ValueError:
            pass
        return value

    def get_db_prep_save(self, value, *args, **kwargs):
        print("get_db_prep_save")
        if value == "":
            return None
        if isinstance(value, dict):
            value = json.dumps(value, cls=DjangoJSONEncoder)
        return super(JSONField, self).get_db_prep_save(value, *args, **kwargs)

In [33]:
class Person3(models.Model):
    extra = JSONField()
    class Meta:
        app_label = "a"

In [34]:
p = Person2()
p.extra = "{'a': 2}"

In [35]:
print(type(p.extra))

<class 'str'>


In [36]:
p.save()

get_db_prep_save
to_python


NameError: name 'json' is not defined